In [2]:
import pandas as pd
import numpy as np

The data file has multiple sheets with different data formats 
- <strain>_ddpcr_data: contains the raw data from the ddPCR machine?
- <strain>: Each row is an experiment
    ex) row 1 is the first generation for that strain in condition D (DMSO), and the coulns are the technical replicates. 

Read in the data file 

In [12]:
#data_file = '/Users/lijiang/Downloads/clay_original_results/clay_results_summary.xlsx'
data_file = 'S14.xlsx'
#read in the data in the 4th sheet of the excel file - 0 indexed
data = pd.read_excel(data_file,sheet_name=6,index_col=False)
#Get the number of rows and columns
rows,cols = data.shape

In [9]:
print(rows,cols)

8 12


For each column the script `clay_fitness.py` executes the function `col_finess()` over that column. Let's get a single example column to see what it does.

In [13]:
range(cols)

range(0, 12)

In [13]:
#pull a single column from the data frame
# data = pd.read_excel('claytest.xlsx',sheet_name=1,index_col=False)
column = data.iloc[:,3]
print(column)

0    0.501814
1    0.576682
2    0.505882
3    0.526523
4    0.533529
5    0.507594
6    0.553981
7    0.515117
Name: replicate_2, dtype: float64


In [14]:
def col_fitness(column):
    array_len = len(column)
    array1=[]
    array2=[]
    x1 = []
    x2 = []
    xs = [0,2,4,6]
    for i in range(array_len):
        if column[i] == -1:
            continue
        if i%2 == 0:
            array1.append(column[i])
            x1.append(xs[i//2])
        else:
            array2.append(column[i])
            x2.append(xs[i//2])
        
    xs = [x1,x2]
    for i,x in enumerate(xs):
        xs[i] = np.array(xs[i])
        xs[i] = xs[i][:,np.newaxis]
    arrays = [array1,array2]
    fitness = []
    for i,array in enumerate(arrays):
        if 'NULL' in array:
            fitness.append('NULL')
        else:
            first_point = array[0]
            ys = []
            for point in array:
                y = np.log10((first_point/point - first_point)/(1-first_point))
                ys.append(y)
            ys = np.array(ys)
            (a, _, _, _) = np.linalg.lstsq(xs[i], ys)
            fitness.append(np.log2(1/np.power(10,a[0])))
    return fitness


In [18]:
col_fitness(column)

/var/folders/h4/rcl4wl655lb_99254fkp0_6r0000gn/T/ipykernel_32104/270481550.py:34: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (a, _, _, _) = np.linalg.lstsq(xs[i], ys)


[0.0023700349740701926, -0.02251498448589777]

In [15]:
array_len = len(column)
array1=[]
array2=[]

x1 = []
x2 = []

#Index of control conditons for the row
xs = [0,2,4,6]

In [16]:
for i in range(array_len):
        #If the at last row, skip
        if column[i] == -1:
            continue
        #If the column is even, add to array1 and x1
        if i%2 == 0:
            #Add a replicates AF for all control contions to array1 for generation 1->7  
            array1.append(column[i])
            #Divide the row
            x1.append(xs[i//2])
        #IF the column is odd, add to array2 and x2
        else:
            #Add a replicates AF for all treated conditions to array2 for generation 1->7
            array2.append(column[i])
            x2.append(xs[i//2])

In [45]:
8//2

4

In [17]:
print(array1)
print(array2)

[0.501813784764208, 0.505882352941176, 0.533529411764705, 0.553980825343893]
[0.576681614349775, 0.526522593320235, 0.507593924860111, 0.515116942384483]


In [18]:
print(x1)
print(x2)
xs = [x1,x2]
print(xs)


[0, 2, 4, 6]
[0, 2, 4, 6]
[[0, 2, 4, 6], [0, 2, 4, 6]]


In [19]:
for i,x in enumerate(xs):
    xs[i] = np.array(xs[i])
    print(xs[i])
    xs[i] = xs[i][:,np.newaxis]
    print(xs[i])


[0 2 4 6]
[[0]
 [2]
 [4]
 [6]]
[0 2 4 6]
[[0]
 [2]
 [4]
 [6]]


In [20]:
arrays = [array1,array2]
print(arrays)

[[0.501813784764208, 0.505882352941176, 0.533529411764705, 0.553980825343893], [0.576681614349775, 0.526522593320235, 0.507593924860111, 0.515116942384483]]


In [21]:
fitness = []

Fitness cal for a single element in the array

In [22]:
#for i,array in enumerate(arrays):
#    print(array)
ex_array = arrays[0]
print(ex_array)


[0.501813784764208, 0.505882352941176, 0.533529411764705, 0.553980825343893]


In [23]:
ys = []

In [24]:
for point in ex_array:            
                first_point = ex_array[0]
                y = np.log10((first_point/point - first_point)/(1-first_point))
                ys.append(y)

In [63]:
print(ys)

[0.0, -0.007068284502727101, -0.05518321906353173, -0.09099032257856374]


In [25]:
ys = np.array(ys)
(a, _, _, _) = np.linalg.lstsq(xs[i], ys)
print((a, _, _, _))

(array([-0.01394306]), array([7.48331477]), array([7.48331477]), array([7.48331477]))


/var/folders/h4/rcl4wl655lb_99254fkp0_6r0000gn/T/ipykernel_35656/2477271447.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (a, _, _, _) = np.linalg.lstsq(xs[i], ys)


In [26]:
np.linalg.lstsq(xs[i], ys)

/var/folders/h4/rcl4wl655lb_99254fkp0_6r0000gn/T/ipykernel_35656/737842882.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.linalg.lstsq(xs[i], ys)


(array([-0.01394306]), array([0.00048749]), 1, array([7.48331477]))

In [75]:
fitness.append(np.log2(1/np.power(10,a[0])))

In [76]:
print(fitness)

[0.04631784397246407]
